In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston = load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df['price'] = boston.target
print(df.shape)
df.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### update weight 구하기
- 많은 특성 중 rm, lstat 두개의 특성에 대해서 weight update를 구한다
- bias_factors는 bias행렬의 초기화를 위해 모든 값이 0인 행렬로 초기화해줌

In [4]:
def get_update_weights(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    N = len(target)
    pred = w1*rm + w2*lstat + bias
    diff = target - pred
    bias_factors = np.ones((N,))
    
    w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))
    
    mse_loss = np.mean(np.square(diff))
    
    return bias_update, w1_update, w2_update, mse_loss

### gradient descent 구현
- 최초 w1, w2, bias를 0으로 초기화해줌
- get_update_weights 함수를 통해 구한 update 값들을 이용하여 값들을 upate시켜줌

In [5]:
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    print('최초 w1, w2, bias: ', w1, w2, bias)
    
    learning_rate = 0.01
    rm = features[:,0]
    lstat = features[:,1]
    
    for i in range(iter_epochs):
        bias_update, w1_update, w2_update, loss = get_update_weights(bias, w1, w2, rm, lstat, target, learning_rate)
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:',w1,'w2:',w2, 'bias:', bias, 'loss:', loss)
    return w1, w2, bias

### 적용
- 신경망은 ```정규화/표준화``` 작업 해줘야함
- MinMaxScaler를 사용하면 feature값들이 0~1 사이 값으로 변환됨

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['RM','LSTAT']])

w1, w2, bias = gradient_descent(scaled_features, df['price'].values, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #####')
print(w1, w2, bias)

최초 w1, w2, bias:  [0.] [0.] [0.]
Epoch: 1 / 5000
w1: [0.252369] w2: [0.10914761] bias: [0.45065613] loss: 592.1469169960474
Epoch: 2 / 5000
w1: [0.4982605] w2: [0.21458377] bias: [0.8890071] loss: 564.6567515182813
Epoch: 3 / 5000
w1: [0.73785103] w2: [0.31641055] bias: [1.315389] loss: 538.6424811965484
Epoch: 4 / 5000
w1: [0.97131229] w2: [0.41472723] bias: [1.73012873] loss: 514.0245946883915
Epoch: 5 / 5000
w1: [1.1988113] w2: [0.50963037] bias: [2.13354428] loss: 490.7278647125017
Epoch: 6 / 5000
w1: [1.42051052] w2: [0.60121392] bias: [2.52594493] loss: 468.6811172230454
Epoch: 7 / 5000
w1: [1.63656797] w2: [0.68956922] bias: [2.90763152] loss: 447.81701302090454
Epoch: 8 / 5000
w1: [1.84713735] w2: [0.77478516] bias: [3.27889669] loss: 428.07184113172934
Epoch: 9 / 5000
w1: [2.05236818] w2: [0.85694818] bias: [3.64002506] loss: 409.3853233168043
Epoch: 10 / 5000
w1: [2.25240586] w2: [0.93614234] bias: [3.9912935] loss: 391.700429116892
Epoch: 11 / 5000
w1: [2.44739187] w2: [1.01

w1: [11.22838648] w2: [0.37039538] bias: [16.40652658] loss: 68.359342690007
Epoch: 182 / 5000
w1: [11.24324696] w2: [0.34800726] bias: [16.40962443] loss: 68.28602156797125
Epoch: 183 / 5000
w1: [11.25805195] w2: [0.32561658] bias: [16.41264017] loss: 68.21290680869545
Epoch: 184 / 5000
w1: [11.27280263] w2: [0.30322399] bias: [16.41557605] loss: 68.13999442884756
Epoch: 185 / 5000
w1: [11.28750014] w2: [0.28083014] bias: [16.41843424] loss: 68.06728064611826
Epoch: 186 / 5000
w1: [11.30214562] w2: [0.25843564] bias: [16.42121685] loss: 67.99476186841567
Epoch: 187 / 5000
w1: [11.31674016] w2: [0.2360411] bias: [16.42392595] loss: 67.9224346836422
Epoch: 188 / 5000
w1: [11.33128482] w2: [0.2136471] bias: [16.42656354] loss: 67.8502958500224
Epoch: 189 / 5000
w1: [11.34578064] w2: [0.19125422] bias: [16.42913157] loss: 67.77834228695171
Epoch: 190 / 5000
w1: [11.36022862] w2: [0.16886301] bias: [16.43163192] loss: 67.70657106633823
Epoch: 191 / 5000
w1: [11.37462974] w2: [0.14647401] b

w1: [13.32839868] w2: [-3.17400079] bias: [16.52895049] loss: 58.07565975321454
Epoch: 348 / 5000
w1: [13.33978139] w2: [-3.19386382] bias: [16.52904749] loss: 58.02317314848441
Epoch: 349 / 5000
w1: [13.35115301] w2: [-3.21370806] bias: [16.52914348] loss: 57.97078657458009
Epoch: 350 / 5000
w1: [13.36251355] w2: [-3.23353352] bias: [16.52923849] loss: 57.91849984048635
Epoch: 351 / 5000
w1: [13.37386303] w2: [-3.25334021] bias: [16.52933254] loss: 57.86631275557178
Epoch: 352 / 5000
w1: [13.38520148] w2: [-3.27312815] bias: [16.52942564] loss: 57.81422512958683
Epoch: 353 / 5000
w1: [13.39652892] w2: [-3.29289734] bias: [16.52951782] loss: 57.76223677266224
Epoch: 354 / 5000
w1: [13.40784536] w2: [-3.3126478] bias: [16.52960911] loss: 57.7103474953072
Epoch: 355 / 5000
w1: [13.41915083] w2: [-3.33237954] bias: [16.52969951] loss: 57.658557108407834
Epoch: 356 / 5000
w1: [13.43044534] w2: [-3.35209258] bias: [16.52978905] loss: 57.606865423225635
Epoch: 357 / 5000
w1: [13.44172893] w2

w1: [15.14501428] w2: [-6.34306792] bias: [16.53904244] loss: 50.33427759635942
Epoch: 521 / 5000
w1: [15.15467908] w2: [-6.35990212] bias: [16.5390806] loss: 50.29654364261581
Epoch: 522 / 5000
w1: [15.16433483] w2: [-6.37672018] bias: [16.5391186] loss: 50.25888159569668
Epoch: 523 / 5000
w1: [15.17398154] w2: [-6.39352211] bias: [16.53915645] loss: 50.221291318554506
Epoch: 524 / 5000
w1: [15.18361922] w2: [-6.41030793] bias: [16.53919414] loss: 50.18377267440295
Epoch: 525 / 5000
w1: [15.19324787] w2: [-6.42707766] bias: [16.53923167] loss: 50.146325526716424
Epoch: 526 / 5000
w1: [15.20286751] w2: [-6.4438313] bias: [16.53926904] loss: 50.10894973922947
Epoch: 527 / 5000
w1: [15.21247814] w2: [-6.46056889] bias: [16.53930625] loss: 50.07164517593643
Epoch: 528 / 5000
w1: [15.22207977] w2: [-6.47729042] bias: [16.53934331] loss: 50.03441170109081
Epoch: 529 / 5000
w1: [15.23167241] w2: [-6.49399593] bias: [16.53938021] loss: 49.99724917920488
Epoch: 530 / 5000
w1: [15.24125606] w2:

Epoch: 700 / 5000
w1: [16.74679738] w2: [-9.12730107] bias: [16.54357324] loss: 44.579932870966196
Epoch: 701 / 5000
w1: [16.75496463] w2: [-9.14146437] bias: [16.54358623] loss: 44.553164265953804
Epoch: 702 / 5000
w1: [16.76312426] w2: [-9.15561408] bias: [16.5435991] loss: 44.52644666893861
Epoch: 703 / 5000
w1: [16.77127628] w2: [-9.16975021] bias: [16.54361183] loss: 44.49977998270506
Epoch: 704 / 5000
w1: [16.77942069] w2: [-9.18387277] bias: [16.54362444] loss: 44.47316411022288
Epoch: 705 / 5000
w1: [16.78755749] w2: [-9.19798178] bias: [16.54363693] loss: 44.44659895464676
Epoch: 706 / 5000
w1: [16.7956867] w2: [-9.21207724] bias: [16.54364929] loss: 44.42008441931593
Epoch: 707 / 5000
w1: [16.80380833] w2: [-9.22615917] bias: [16.54366153] loss: 44.393620407753865
Epoch: 708 / 5000
w1: [16.81192237] w2: [-9.24022759] bias: [16.54367364] loss: 44.36720682366794
Epoch: 709 / 5000
w1: [16.82002884] w2: [-9.25428251] bias: [16.54368563] loss: 44.34084357094904
Epoch: 710 / 5000
w

...

Epoch: 4268 / 5000
w1: [25.39130543] w2: [-23.28826718] bias: [16.30519126] loss: 30.538140004434737
Epoch: 4269 / 5000
w1: [25.39169396] w2: [-23.28867382] bias: [16.30511074] loss: 30.538107681895884
Epoch: 4270 / 5000
w1: [25.39208221] w2: [-23.28908002] bias: [16.30503021] loss: 30.538075416505258
Epoch: 4271 / 5000
w1: [25.3924702] w2: [-23.28948578] bias: [16.3049497] loss: 30.538043208155123
Epoch: 4272 / 5000
w1: [25.39285792] w2: [-23.28989109] bias: [16.30486919] loss: 30.538011056737965
Epoch: 4273 / 5000
w1: [25.39324538] w2: [-23.29029596] bias: [16.30478868] loss: 30.53797896214648
Epoch: 4274 / 5000
w1: [25.39363256] w2: [-23.29070039] bias: [16.30470819] loss: 30.537946924273545
Epoch: 4275 / 5000
w1: [25.39401948] w2: [-23.29110438] bias: [16.3046277] loss: 30.53791494301225
Epoch: 4276 / 5000
w1: [25.39440612] w2: [-23.29150793] bias: [16.30454722] loss: 30.537883018255894
Epoch: 4277 / 5000
w1: [25.3947925] w2: [-23.29191103] bias: [16.30446674] loss: 30.537851149897

w1: [25.45849973] w2: [-23.35604445] bias: [16.29048942] loss: 30.533053635613822
Epoch: 4453 / 5000
w1: [25.45884258] w2: [-23.35637634] bias: [16.29041017] loss: 30.533030207182787
Epoch: 4454 / 5000
w1: [25.45918521] w2: [-23.35670786] bias: [16.29033093] loss: 30.53300681916304
Epoch: 4455 / 5000
w1: [25.45952761] w2: [-23.357039] bias: [16.29025169] loss: 30.532983471478712
Epoch: 4456 / 5000
w1: [25.45986978] w2: [-23.35736978] bias: [16.29017246] loss: 30.532960164054042
Epoch: 4457 / 5000
w1: [25.46021173] w2: [-23.35770019] bias: [16.29009324] loss: 30.53293689681345
Epoch: 4458 / 5000
w1: [25.46055345] w2: [-23.35803023] bias: [16.29001402] loss: 30.532913669681484
Epoch: 4459 / 5000
w1: [25.46089494] w2: [-23.35835989] bias: [16.28993481] loss: 30.532890482582832
Epoch: 4460 / 5000
w1: [25.4612362] w2: [-23.35868919] bias: [16.28985561] loss: 30.53286733544233
Epoch: 4461 / 5000
w1: [25.46157724] w2: [-23.35901813] bias: [16.28977642] loss: 30.532844228184956
Epoch: 4462 / 5

w1: [25.51763979] w2: [-23.41095212] bias: [16.27610823] loss: 30.52936906386868
Epoch: 4636 / 5000
w1: [25.51794411] w2: [-23.41122182] bias: [16.27603033] loss: 30.529351900395607
Epoch: 4637 / 5000
w1: [25.51824824] w2: [-23.41149122] bias: [16.27595244] loss: 30.5293347655965
Epoch: 4638 / 5000
w1: [25.51855217] w2: [-23.4117603] bias: [16.27587455] loss: 30.52931765941777
Epoch: 4639 / 5000
w1: [25.51885591] w2: [-23.41202908] bias: [16.27579667] loss: 30.52930058180596
Epoch: 4640 / 5000
w1: [25.51915945] w2: [-23.41229754] bias: [16.2757188] loss: 30.529283532707698
Epoch: 4641 / 5000
w1: [25.51946281] w2: [-23.4125657] bias: [16.27564094] loss: 30.529266512069732
Epoch: 4642 / 5000
w1: [25.51976596] w2: [-23.41283355] bias: [16.27556308] loss: 30.529249519838896
Epoch: 4643 / 5000
w1: [25.52006893] w2: [-23.41310109] bias: [16.27548523] loss: 30.529232555962132
Epoch: 4644 / 5000
w1: [25.52037171] w2: [-23.41336832] bias: [16.27540739] loss: 30.529215620386477
Epoch: 4645 / 500

w1: [25.57108254] w2: [-23.45608596] bias: [16.26175024] loss: 30.526618059746216
Epoch: 4822 / 5000
w1: [25.57135361] w2: [-23.45630288] bias: [16.26167377] loss: 30.52660540668505
Epoch: 4823 / 5000
w1: [25.57162451] w2: [-23.45651954] bias: [16.26159731] loss: 30.526592773897914
Epoch: 4824 / 5000
w1: [25.57189524] w2: [-23.45673595] bias: [16.26152086] loss: 30.52658016134719
Epoch: 4825 / 5000
w1: [25.57216581] w2: [-23.4569521] bias: [16.26144441] loss: 30.526567568995354
Epoch: 4826 / 5000
w1: [25.57243622] w2: [-23.45716799] bias: [16.26136798] loss: 30.52655499680494
Epoch: 4827 / 5000
w1: [25.57270646] w2: [-23.45738362] bias: [16.26129155] loss: 30.526542444738535
Epoch: 4828 / 5000
w1: [25.57297654] w2: [-23.45759899] bias: [16.26121512] loss: 30.526529912758832
Epoch: 4829 / 5000
w1: [25.57324646] w2: [-23.45781411] bias: [16.26113871] loss: 30.526517400828567
Epoch: 4830 / 5000
w1: [25.57351621] w2: [-23.45802897] bias: [16.2610623] loss: 30.526504908910557
Epoch: 4831 / 

w1: [25.61686884] w2: [-23.49084945] bias: [16.24826322] loss: 30.52465313914625
Epoch: 5000 / 5000
w1: [25.61711278] w2: [-23.49102413] bias: [16.24818816] loss: 30.524643563228903
##### 최종 w1, w2, bias #####
[25.61711278] [-23.49102413] [16.24818816]


### 계산된 weight, bias로 price 예측

In [7]:
pred = scaled_features[:,0]*w1 + scaled_features[:,1]*w2 + bias
df['predicted_price'] = pred
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price,predicted_price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.935533
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.483093
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.545474
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.334142
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.516284


### keras 사용
- 회귀의 경우 activation 함수가 필요없다
- units는 hidden layer의 node의 수를 정하는 것

In [9]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(scaled_features, df['price'].values, epochs=5000)

Epoch 1/5000
16/16 [==============================] - 0s 667us/step - loss: 544.0526 - mse: 544.0526
Epoch 2/5000
16/16 [==============================] - 0s 601us/step - loss: 523.4806 - mse: 523.4806
Epoch 3/5000
16/16 [==============================] - 0s 601us/step - loss: 527.0675 - mse: 527.0675
Epoch 4/5000
16/16 [==============================] - 0s 601us/step - loss: 519.3724 - mse: 519.3724
Epoch 5/5000
16/16 [==============================] - 0s 534us/step - loss: 510.8400 - mse: 510.8400
Epoch 6/5000
16/16 [==============================] - 0s 601us/step - loss: 481.6807 - mse: 481.6807
Epoch 7/5000
16/16 [==============================] - 0s 601us/step - loss: 475.9724 - mse: 475.9724
Epoch 8/5000
16/16 [==============================] - 0s 601us/step - loss: 467.1936 - mse: 467.1936
Epoch 9/5000
16/16 [==============================] - 0s 534us/step - loss: 435.2810 - mse: 435.2810
Epoch 10/5000
16/16 [==============================] - 0s 601us/step - loss: 427.6927 - mse

16/16 [==============================] - 0s 534us/step - loss: 125.6456 - mse: 125.6456
Epoch 82/5000
16/16 [==============================] - 0s 601us/step - loss: 117.0210 - mse: 117.0210
Epoch 83/5000
16/16 [==============================] - 0s 601us/step - loss: 103.2384 - mse: 103.2384
Epoch 84/5000
16/16 [==============================] - 0s 534us/step - loss: 118.6951 - mse: 118.6951
Epoch 85/5000
16/16 [==============================] - 0s 534us/step - loss: 124.0752 - mse: 124.0752
Epoch 86/5000
16/16 [==============================] - 0s 534us/step - loss: 115.2198 - mse: 115.2198
Epoch 87/5000
16/16 [==============================] - 0s 534us/step - loss: 105.8801 - mse: 105.8801
Epoch 88/5000
16/16 [==============================] - 0s 534us/step - loss: 91.9119 - mse: 91.9119
Epoch 89/5000
16/16 [==============================] - 0s 534us/step - loss: 112.7379 - mse: 112.7379
Epoch 90/5000
16/16 [==============================] - 0s 534us/step - loss: 102.6634 - mse: 102.6

16/16 [==============================] - 0s 601us/step - loss: 68.1353 - mse: 68.1353
Epoch 163/5000
16/16 [==============================] - 0s 601us/step - loss: 73.9307 - mse: 73.9307
Epoch 164/5000
16/16 [==============================] - 0s 534us/step - loss: 61.0777 - mse: 61.0777
Epoch 165/5000
16/16 [==============================] - 0s 534us/step - loss: 69.0082 - mse: 69.0082
Epoch 166/5000
16/16 [==============================] - 0s 600us/step - loss: 68.2045 - mse: 68.2045
Epoch 167/5000
16/16 [==============================] - 0s 601us/step - loss: 64.0371 - mse: 64.0371
Epoch 168/5000
16/16 [==============================] - 0s 534us/step - loss: 66.9775 - mse: 66.9775
Epoch 169/5000
16/16 [==============================] - 0s 601us/step - loss: 62.2891 - mse: 62.2891
Epoch 170/5000
16/16 [==============================] - 0s 601us/step - loss: 72.4251 - mse: 72.4251
Epoch 171/5000
16/16 [==============================] - 0s 534us/step - loss: 69.0929 - mse: 69.0929
Epoch

16/16 [==============================] - 0s 534us/step - loss: 51.4880 - mse: 51.4880
Epoch 244/5000
16/16 [==============================] - 0s 667us/step - loss: 47.7438 - mse: 47.7438
Epoch 245/5000
16/16 [==============================] - 0s 601us/step - loss: 44.4248 - mse: 44.4248
Epoch 246/5000
16/16 [==============================] - 0s 667us/step - loss: 43.3362 - mse: 43.3362
Epoch 247/5000
16/16 [==============================] - 0s 603us/step - loss: 48.1953 - mse: 48.1953
Epoch 248/5000
16/16 [==============================] - 0s 585us/step - loss: 44.5044 - mse: 44.5044
Epoch 249/5000
16/16 [==============================] - 0s 601us/step - loss: 41.9891 - mse: 41.9891
Epoch 250/5000
16/16 [==============================] - 0s 667us/step - loss: 48.6995 - mse: 48.6995
Epoch 251/5000
16/16 [==============================] - 0s 601us/step - loss: 47.4629 - mse: 47.4629
Epoch 252/5000
16/16 [==============================] - 0s 534us/step - loss: 49.8464 - mse: 49.8464
Epoch

16/16 [==============================] - 0s 601us/step - loss: 38.5679 - mse: 38.5679
Epoch 324/5000
16/16 [==============================] - 0s 667us/step - loss: 39.0811 - mse: 39.0811
Epoch 325/5000
16/16 [==============================] - 0s 534us/step - loss: 38.1587 - mse: 38.1587
Epoch 326/5000
16/16 [==============================] - 0s 667us/step - loss: 29.6648 - mse: 29.6648
Epoch 327/5000
16/16 [==============================] - 0s 534us/step - loss: 36.5893 - mse: 36.5893
Epoch 328/5000
16/16 [==============================] - 0s 534us/step - loss: 29.2283 - mse: 29.2283
Epoch 329/5000
16/16 [==============================] - 0s 601us/step - loss: 28.1943 - mse: 28.1943
Epoch 330/5000
16/16 [==============================] - 0s 601us/step - loss: 36.4145 - mse: 36.4145
Epoch 331/5000
16/16 [==============================] - 0s 534us/step - loss: 29.8853 - mse: 29.8853
Epoch 332/5000
16/16 [==============================] - ETA: 0s - loss: 17.9416 - mse: 17.941 - 0s 534us/s

...

16/16 [==============================] - 0s 534us/step - loss: 34.0268 - mse: 34.0268
Epoch 4645/5000
16/16 [==============================] - 0s 534us/step - loss: 29.1987 - mse: 29.1987
Epoch 4646/5000
16/16 [==============================] - 0s 534us/step - loss: 30.0561 - mse: 30.0561
Epoch 4647/5000
16/16 [==============================] - 0s 601us/step - loss: 28.1171 - mse: 28.1171
Epoch 4648/5000
16/16 [==============================] - 0s 534us/step - loss: 28.7316 - mse: 28.7316
Epoch 4649/5000
16/16 [==============================] - 0s 601us/step - loss: 26.4277 - mse: 26.4277
Epoch 4650/5000
16/16 [==============================] - 0s 601us/step - loss: 31.5504 - mse: 31.5504
Epoch 4651/5000
16/16 [==============================] - 0s 534us/step - loss: 27.0579 - mse: 27.0579
Epoch 4652/5000
16/16 [==============================] - 0s 534us/step - loss: 28.6103 - mse: 28.6103
Epoch 4653/5000
16/16 [==============================] - 0s 667us/step - loss: 27.5551 - mse: 27.5

16/16 [==============================] - 0s 534us/step - loss: 31.9293 - mse: 31.9293
Epoch 4725/5000
16/16 [==============================] - 0s 601us/step - loss: 31.1448 - mse: 31.1448
Epoch 4726/5000
16/16 [==============================] - 0s 534us/step - loss: 29.6129 - mse: 29.6129
Epoch 4727/5000
16/16 [==============================] - 0s 601us/step - loss: 28.3209 - mse: 28.3209
Epoch 4728/5000
16/16 [==============================] - 0s 534us/step - loss: 33.5421 - mse: 33.5421
Epoch 4729/5000
16/16 [==============================] - 0s 534us/step - loss: 30.8260 - mse: 30.8260
Epoch 4730/5000
16/16 [==============================] - 0s 600us/step - loss: 33.7289 - mse: 33.7289
Epoch 4731/5000
16/16 [==============================] - 0s 534us/step - loss: 30.1963 - mse: 30.1963
Epoch 4732/5000
16/16 [==============================] - 0s 534us/step - loss: 31.6014 - mse: 31.6014
Epoch 4733/5000
16/16 [==============================] - 0s 601us/step - loss: 26.7263 - mse: 26.7

16/16 [==============================] - 0s 534us/step - loss: 32.8066 - mse: 32.8066
Epoch 4805/5000
16/16 [==============================] - 0s 534us/step - loss: 28.3759 - mse: 28.3759
Epoch 4806/5000
16/16 [==============================] - 0s 601us/step - loss: 31.7068 - mse: 31.7068
Epoch 4807/5000
16/16 [==============================] - 0s 534us/step - loss: 30.8509 - mse: 30.8509
Epoch 4808/5000
16/16 [==============================] - 0s 601us/step - loss: 32.3635 - mse: 32.3635
Epoch 4809/5000
16/16 [==============================] - 0s 600us/step - loss: 30.5601 - mse: 30.5601
Epoch 4810/5000
16/16 [==============================] - 0s 534us/step - loss: 25.3768 - mse: 25.3768
Epoch 4811/5000
16/16 [==============================] - 0s 601us/step - loss: 29.5812 - mse: 29.5812
Epoch 4812/5000
16/16 [==============================] - 0s 467us/step - loss: 28.8898 - mse: 28.8898
Epoch 4813/5000
16/16 [==============================] - 0s 534us/step - loss: 34.1837 - mse: 34.1

16/16 [==============================] - 0s 534us/step - loss: 29.3900 - mse: 29.3900
Epoch 4885/5000
16/16 [==============================] - 0s 534us/step - loss: 32.7155 - mse: 32.7155
Epoch 4886/5000
16/16 [==============================] - 0s 534us/step - loss: 32.3847 - mse: 32.3847
Epoch 4887/5000
16/16 [==============================] - 0s 601us/step - loss: 32.8821 - mse: 32.8821
Epoch 4888/5000
16/16 [==============================] - 0s 601us/step - loss: 27.7691 - mse: 27.7691
Epoch 4889/5000
16/16 [==============================] - 0s 534us/step - loss: 30.4539 - mse: 30.4539
Epoch 4890/5000
16/16 [==============================] - 0s 601us/step - loss: 32.4642 - mse: 32.4642
Epoch 4891/5000
16/16 [==============================] - 0s 534us/step - loss: 33.0070 - mse: 33.0070
Epoch 4892/5000
16/16 [==============================] - 0s 601us/step - loss: 28.7555 - mse: 28.7555
Epoch 4893/5000
16/16 [==============================] - 0s 601us/step - loss: 35.4686 - mse: 35.4

16/16 [==============================] - 0s 534us/step - loss: 33.9025 - mse: 33.9025
Epoch 4965/5000
16/16 [==============================] - 0s 534us/step - loss: 34.3758 - mse: 34.3758
Epoch 4966/5000
16/16 [==============================] - 0s 601us/step - loss: 32.4609 - mse: 32.4609
Epoch 4967/5000
16/16 [==============================] - 0s 534us/step - loss: 30.0902 - mse: 30.0902
Epoch 4968/5000
16/16 [==============================] - 0s 601us/step - loss: 27.7163 - mse: 27.7163
Epoch 4969/5000
16/16 [==============================] - 0s 601us/step - loss: 29.2004 - mse: 29.2004
Epoch 4970/5000
16/16 [==============================] - 0s 601us/step - loss: 30.5915 - mse: 30.5915
Epoch 4971/5000
16/16 [==============================] - 0s 734us/step - loss: 31.3798 - mse: 31.3798
Epoch 4972/5000
16/16 [==============================] - 0s 667us/step - loss: 29.5171 - mse: 29.5171
Epoch 4973/5000
16/16 [==============================] - 0s 601us/step - loss: 25.6974 - mse: 25.6

- 앞서 model.fit 을 하게 되면 각 feature들에 대한 적절한 weight값으로 채워지게 됨
- 이를 이용해 scaling 한 데이터 값을 model.predict 하게 되면 계산한 weight 값을 이용하여 계산

In [10]:
pred = model.predict(scaled_features)
df['keras_predicted_price'] = pred
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price,predicted_price,keras_predicted_price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.935533,28.974293
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.483093,25.521244
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.545474,32.692596
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.334142,32.438622
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.516284,31.665123
